In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed

In [2]:
# root_dir = Path("../data_inputs/goodwin_creek_1m/")
root_dir = Path(
    "../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach"
)
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"
path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

In [3]:
watershed = Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()
watershed.ignore_reaches_with_strahler_leq(0)

# watershed.identify_inflow_sources()
# watershed.update_junctions_and_node_types()

# watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
# watershed.set_node_id_to_compute_id()
watershed.update_watershed()

watershed.resample_reaches(id_list="all", numpoints=10)

Reading Reaches: 100%|██████████| 3/3 [00:00<00:00, 79.50it/s]


In [4]:
reaches = watershed.reaches
for k, v in reaches.items():
    # if not v.ignore:
    print(v)
# break

-----------------------------
Reach            : 2
CCHE1D Ch. ID    : 3
Receiving Reach  : 1
Slope            : 0.01716
Strahler Number  : 2
Ignore?          : False
(US/DS) Nodes    : (23, 32)
-----------------------------
Reach            : 3
CCHE1D Ch. ID    : 2
Receiving Reach  : 2
Slope            : 0.01082
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (12, 22)
-----------------------------
Reach            : 4
CCHE1D Ch. ID    : 1
Receiving Reach  : 2
Slope            : 0.01332
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (1, 11)


In [5]:
reach2 = watershed.reaches[2]
reach3 = watershed.reaches[3]
reach4 = watershed.reaches[4]

In [10]:
print(reach2.nodes[reach2.us_nd_id])

-----------------------------
Node ID         : 23
TYPE            : 2
USID            : 22
DSID            : 24
US2ID           : 11
COMPUTEID       : 23
Cell Source     : None
Reach Sources   : []
(x,y)           : (240167.114013, 3795272.620724)
(row,col)       : (None, None)


In [11]:
print(reach3.nodes[reach3.ds_nd_id])

-----------------------------
Node ID         : 22
TYPE            : 3
USID            : 21
DSID            : 23
US2ID           : -1
COMPUTEID       : 22
Cell Source     : None
Reach Sources   : []
(x,y)           : (240167.114013, 3795272.620724)
(row,col)       : (None, None)


## Plotting network

In [6]:
watershed.plot(
    frame_height=700, frame_width=1200, line_width=2, by="Reach_ID", title="Watershed"
)

:Overlay
   .NdOverlay.I  :NdOverlay   [Reach_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [Reach_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

In [7]:
import holoviews as hv
import numpy as np
from scipy.interpolate import make_interp_spline
from math import floor, ceil

In [8]:
u = [1, 2, 3, 4]
v = ["a", "b", "c", "d"]

for c, (ui, vi) in enumerate(zip(u, v), 3):
    print(c, ui, vi)

3 1 a
4 2 b
5 3 c
6 4 d


In [9]:
max(1, None)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [ ]:
def interpolate_xy_cord_step_numpoints(x, y, **kwargs):
    """
    Given two x and y arrays provides two new arrays xnew, ynew that are interpolated:
    key-value arguments:
        - step : define a step length (in the units of x and y) to resample points along the cord length
        OR
        - numpoints: define an arbitrary number of points to resample along the cord length
    WARNING: If both values are specified, the method yielding the greater of resampling nodes will be chosen
    It is advised to use only one of the keywords arguments.
    """
    if "step" in kwargs:
        step = kwargs["step"]
    else:
        step = None

    if "numpoints" in kwargs:
        numpoints = kwargs["numpoints"]
    else:
        numpoints = 0

    if step is None and numpoints == 0:
        raise Exception("No resampling parameter provided (step, or numpoints)")

    # Arrange coordinates in pairs:
    p = np.stack((x, y))

    dp = p[:, 1:] - p[:, :-1]  # 2 vector distance between points
    l = (dp**2).sum(axis=0)  # squares of lengths of 2-vectors between points
    u_cord = np.sqrt(l).cumsum()  # cumulative sums of 2-norms
    u_cord = np.r_[0, u_cord]  # the first point is parameterized at zero

    # Get arc length
    arc_length = u_cord[-1]  # by definition

    # Compute number of nodes needed for interpolation
    if step is not None:
        numpoints = max(
            ceil(arc_length / step), numpoints
        )  # chooses the highest value between
        # provided numpoints and the one computed
        # with the step method
    # create spline interpolant
    spl_cord = make_interp_spline(u_cord, np.c_[x, y])

    uu = np.linspace(u_cord[0], u_cord[-1], numpoints)
    xx_cord, yy_cord = spl_cord(uu).T

    return xx_cord, yy_cord

In [ ]:
x, y = reach2.get_x_y_node_arrays_us_ds_order()
scatter_line_og = hv.Scatter((x, y), label="original").opts(
    aspect="equal", size=8, frame_width=1200
) * hv.Curve((x, y))

# interp
p = np.stack((x, y))
dp = p[:, 1:] - p[:, :-1]  # 2 vector distance between points
l = (dp**2).sum(axis=0)  # squares of lengths of 2-vectors between points
u_cord = np.sqrt(l).cumsum()  # cumulative sums of 2-norms
u_cord = np.r_[0, u_cord]  # the first point is parameterized at zero
u_c = np.r_[0, np.cumsum((dp**2).sum(axis=0) ** 0.25)]  # centripetal

spl_cord = make_interp_spline(u_cord, np.c_[x, y])
# spl_cord = interp1d(u_cord, p)
# spl_c = make_interp_spline(u_c, p, axis=1)    # note p is a 2D array
spl_c = make_interp_spline(u_c, np.c_[x, y])  # note p is a 2D array

# uu = np.linspace(u_cord, p, 51)

numpoints = 5

arc_length = u_cord[-1]
step = 0.1

numpoints = floor(arc_length / step)


uu = np.linspace(u_cord[0], u_cord[-1], numpoints)
xx_cord, yy_cord = spl_cord(uu).T

uu = np.linspace(u_c[0], u_c[-1], numpoints)
xx_c, yy_c = spl_c(uu).T

:Overlay
   .Scatter.Original    :Scatter   [x]   (y)
   .Curve.I             :Curve   [x]   (y)
   .Scatter.Cord        :Scatter   [x]   (y)
   .Curve.II            :Curve   [x]   (y)
   .Scatter.Centripetal :Scatter   [x]   (y)
   .Curve.III           :Curve   [x]   (y)

In [ ]:
x, y = reach2.get_x_y_node_arrays_us_ds_order()

xx_cord, yy_cord = interpolate_xy_cord_step_numpoints(x, y, numpoints=21)

In [ ]:
scatter_line_cord = hv.Scatter((xx_cord, yy_cord), label="cord").opts(
    aspect="equal", size=8
) * hv.Curve((xx_cord, yy_cord))
# scatter_line_c = (hv.Scatter((xx_c, yy_c), label='centripetal').opts(aspect='equal', size=8) * hv.Curve((xx_c,yy_c)))

# scatter_line_og

# scatter_line_c * scatter_line_cord

# (scatter_line_og * scatter_line_cord * scatter_line_c).opts(tools=['hover'])
(scatter_line_og * scatter_line_cord).opts(tools=["hover"])

:Overlay
   .Scatter.Original :Scatter   [x]   (y)
   .Curve.I          :Curve   [x]   (y)
   .Scatter.Cord     :Scatter   [x]   (y)
   .Curve.II         :Curve   [x]   (y)

In [ ]:
arc_length

38.79241716360384

In [ ]:
L = [1, 2, 3, 4, 5, 6, 7]
print(L[::-1])

[7, 6, 5, 4, 3, 2, 1]


In [ ]:
df2 = reach2.get_nodes_as_df()

df2.hvplot.line(x="X", y="Y", by="Strahler_Number", aspect="equal", line_width=2)

:NdOverlay   [Strahler_Number]
   :Curve   [X]   (Y)